In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
BASE_DIR = "C:\\Users\\Mohamadreza.a\\Downloads\\course_scheduling"

In [3]:
# reading sets 

In [4]:
sets_data= pd.read_excel(
     io = os.path.join(BASE_DIR, "assg_3.xlsx"),
     sheet_name = 'Sets',
     engine='openpyxl'
)


In [5]:
p_dataframe = sets_data['p']
p_dataframe = p_dataframe.dropna()
p_dataframe = p_dataframe.astype(int)

c_dataframe = sets_data['c']
c_dataframe = c_dataframe.dropna()
c_dataframe = c_dataframe.astype(int)

j_dataframe = sets_data['j']
j_dataframe = j_dataframe.dropna()
j_dataframe = j_dataframe.astype(int)

d_dataframe = sets_data['d']
d_dataframe = d_dataframe.dropna()
d_dataframe = d_dataframe.astype(int)

h_dataframe = sets_data['h']
h_dataframe = h_dataframe.dropna()
h_dataframe = h_dataframe.astype(int)


In [6]:
import gams
ws = gams.workspace.GamsWorkspace(working_directory = BASE_DIR)
db = ws.add_database()

In [7]:
# adding sets to workspace

In [8]:
j_array = j_dataframe.to_numpy(dtype = str)
j_set = db.add_set('j', 1)
for jp in j_array:
    j_set.add_record(jp)

c_array = c_dataframe.to_numpy(dtype = str)   
c_set = db.add_set('c', 1)
for cp in c_array:
    c_set.add_record(cp)

p_array = p_dataframe.to_numpy(dtype = str)    
p_set = db.add_set('p', 1)
for pp in p_array:
    p_set.add_record(pp)

d_array = d_dataframe.to_numpy(dtype = str)    
d_set = db.add_set('d', 1)
for dp in d_array:
    d_set.add_record(dp)
    

h_array = h_dataframe.to_numpy(dtype = str)
h_set = db.add_set('h', 1)
for hp in h_array:
    h_set.add_record(hp)
    



In [9]:
# read parameters and add to workspace

In [10]:
parameter_a = np.zeros(shape = (len(c_array),len(p_array)))
parameter_a_dict = {} 
parameter_a_data = pd.read_excel(
     io = os.path.join(BASE_DIR, "assg_3.xlsx"),
     sheet_name = 'A',
     engine='openpyxl'
)
p_indices = parameter_a_data['p']
p_indices = p_indices.dropna()
p_indices = p_indices.astype(str).to_numpy()

c_indices = parameter_a_data['c']
c_indices = c_indices.dropna()
c_indices = c_indices.astype(str).to_numpy()

a_data = parameter_a_data['a(c,p)']
a_data = a_data.dropna()
a_data = a_data.astype(str).to_numpy()

for c_index, p_index, correlated_data in zip(c_indices, p_indices, a_data):
    parameter_a[int(c_index)-1][int(p_index)-1] = float(correlated_data)         

for i in range(parameter_a.shape[0]):
    for j in range(parameter_a.shape[1]):
        parameter_a_dict.setdefault( (str(i+1), str(j+1) ), parameter_a[i][j])

In [11]:
a = db.add_parameter_dc('a', [c_set , p_set])
for cp in c_array:
    for pp in p_array :
        a.add_record((cp, pp)).value = parameter_a_dict[(cp,pp)]

In [12]:
parameter_t = np.zeros(shape = (len(h_array),len(h_array)))
parameter_t_dict = {} 
parameter_t_data = pd.read_excel(
     io = os.path.join(BASE_DIR, "assg_3.xlsx"),
     sheet_name = 'T',
     engine='openpyxl'
)
h_indices = parameter_t_data['h']
h_indices = h_indices.dropna()
h_indices = h_indices.astype(str).to_numpy()

h_prime_indices = parameter_t_data['h_prime']
h_prime_indices = h_prime_indices.dropna()
h_prime_indices = h_prime_indices.astype(str).to_numpy()


t_data = parameter_t_data['t(h,h_prime)']
t_data = t_data.dropna()
t_data = t_data.astype(str).to_numpy()

for h_index, h_prime_index, correlated_data in zip(h_indices, h_prime_indices, t_data):
    parameter_t[int(h_index)-1][int(h_index)-1] = int(correlated_data)         

for i in range(parameter_t.shape[0]):
    for j in range(parameter_t.shape[1]):
        parameter_t_dict.setdefault( (str(i+1), str(j+1) ), parameter_t[i][j])

In [13]:
t=db.add_parameter_dc('t',[h_set , h_set])
for hp in h_array:
    for hpp in h_array :
        t.add_record((hp, hpp)).value = parameter_t_dict[(hp,hpp)]

In [14]:
parameter_n = np.zeros(shape = len(c_array))
parameter_n_dict = {} 
parameter_n_data = pd.read_excel(
     io = os.path.join(BASE_DIR, "assg_3.xlsx"),
     sheet_name = 'N',
     engine='openpyxl'
)
c_indices = parameter_n_data['c']
c_indices = c_indices.dropna()
c_indices = c_indices.astype(str).to_numpy()


n_data = parameter_n_data['n(c)']
n_data = n_data.dropna()
n_data = n_data.astype(str).to_numpy()

for c_index, correlated_data in zip(c_indices, n_data):
    parameter_n[int(c_index)-1] = int(correlated_data)         

for i in range(parameter_n.shape[0]):
    parameter_n_dict.setdefault(str(i+1), parameter_n[i])

In [15]:
n = db.add_parameter_dc('n', [c_set])
for cp in c_array:
    n.add_record(cp).value = parameter_n_dict[cp]

In [16]:
parameter_s = np.zeros(shape = (len(j_array),len(c_array)))
parameter_s_dict = {} 
parameter_s_data = pd.read_excel(
     io = os.path.join(BASE_DIR, "assg_3.xlsx"),
     sheet_name = 'S',
     engine='openpyxl'
)
j_indices = parameter_s_data['j']
j_indices = j_indices.dropna()
j_indices = j_indices.astype(str).to_numpy()

c_indices = parameter_s_data['c']
c_indices = c_indices.dropna()
c_indices = c_indices.astype(str).to_numpy()


s_data = parameter_s_data['s(j,c)']
s_data = s_data.dropna()
s_data = s_data.astype(str).to_numpy()

for j_index, c_index, correlated_data in zip(j_indices, c_indices, s_data):
    parameter_s[int(j_index)-1][int(c_index)-1] = int(correlated_data)         

for i in range(parameter_s.shape[0]):
    for j in range(parameter_s.shape[1]):
        parameter_s_dict.setdefault( (str(i+1), str(j+1) ), parameter_s[i][j])

In [17]:
s = db.add_parameter_dc('s', [j_set , c_set])
for jp in j_array:
    for cp in c_array :
        s.add_record((jp, cp)).value = parameter_s_dict[(jp,cp)]

In [18]:
parameter_b = np.zeros(shape = (len(p_array),len(d_array),len(h_array)))
parameter_b_dict = {} 
parameter_b_data = pd.read_excel(
     io = os.path.join(BASE_DIR, "assg_3.xlsx"),
     sheet_name = 'B',
     engine='openpyxl'
)
p_indices = parameter_b_data['p']
p_indices = p_indices.dropna()
p_indices = p_indices.astype(str).to_numpy()

d_indices = parameter_b_data['d']
d_indices = d_indices.dropna()
d_indices = d_indices.astype(str).to_numpy()

h_indices = parameter_b_data['h']
h_indices = h_indices.dropna()
h_indices = h_indices.astype(str).to_numpy()


b_data = parameter_b_data['b(p,d,h)']
b_data = b_data.dropna()
b_data = b_data.astype(str).to_numpy()

for p_index, d_index, h_index, correlated_data in zip(p_indices, d_indices, h_indices, b_data):
    parameter_b[int(p_index)-1][int(d_index)-1][int(h_index)-1] = int(correlated_data)         

for i in range(parameter_b.shape[0]):
    for j in range(parameter_b.shape[1]):
        for k in range(parameter_b.shape[2]):
            parameter_b_dict.setdefault( (str(i+1), str(j+1), (str(k+1) )), parameter_b[i][j][k])

In [19]:
b = db.add_parameter_dc('b', [p_set,d_set,h_set])
for pp in p_array:
    for dp in d_array:
        for hp in h_array :
            b.add_record((pp,dp,hp)).value = parameter_b_dict[(pp,dp,hp)]
    


In [20]:
#define scalars and add to workspace

In [22]:
scalar = db.add_set('sca', 1)
scalar.add_record(str(1))

In [23]:
k = db.add_parameter_dc('k', [scalar])
k.add_record(str(1)).value = 2

dlast = db.add_parameter_dc('dlast', [scalar])
dlast.add_record(str(1)).value = 5
             
hbar = db.add_parameter_dc('hbar', [scalar])
hbar.add_record(str(1)).value = 3

In [24]:
db.export('C:\\Users\\Mohamadreza.a\\Downloads\\course_scheduling\\data.gdx')

In [48]:
model = ws.add_job_from_file(os.path.join(BASE_DIR,"course_scheduling_final.gms"))
model.run()

GamsExceptionExecution: GAMS return code not 0 (7), check C:\Users\Lenovo\Documents\GAMSStudio\workspace\course_scheduling_project\_gams_py_gjo0.lst for more details

In [49]:
for rec in model.out_db['z']:
    print(rec.level)

TypeError: 'NoneType' object is not subscriptable

In [50]:
for rec in model.out_db['x']:
    print(rec.key(0),"\t", rec.key(1),"\t", rec.key(2),"\t",rec.level)

TypeError: 'NoneType' object is not subscriptable

In [51]:
for rec in model.out_db['w']:
    print(rec.key(0),"\t",rec.level)

TypeError: 'NoneType' object is not subscriptable

In [52]:
for rec in model.out_db['v']:
    print(rec.key(0),"\t", rec.key(1),"\t",rec.level)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
for rec in model.out_db['y']:
    print(rec.key(0),"\t",rec.level)